<table align="center">
   <td align="center"><a target="_blank" href="https://colab.research.google.com/github/ds5110/summer-2021/blob/master/08c-diamonds.ipynb">
<img src="https://github.com/ds5110/summer-2021/raw/master/colab.png"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
</table>

# 8c -- diamonds

Linear regression with the diamonds dataset (continued)

In module 6, we covered:

* Visualizing regression models with Seaborn
* Linear regression with linear model
* Nonlinearity (linear regression with polynomials)
* Scaling -- price vs carats

In this module, we'll use the same dataset to cover:

* Using residuals to assess statistical assumptions
* Investigate distributions with histograms to distinguish signal vs noise
* Q-Q plots -- visual assessment of data/model distributions
* Box plots (justifying multiple input/feature model)
* Adding categorical features (forward selection)

## diamonds dataset

* [diamonds dataset](https://ggplot2.tidyverse.org/reference/diamonds.html) -- tidyverse.org



In [ ]:
import pandas as pd

url = "https://github.com/tidyverse/ggplot2/raw/master/data-raw/diamonds.csv"
diamonds = pd.read_csv(url)

diamonds

In [ ]:
import matplotlib.pyplot as plt

# Use opacity (alpha channel) to aid visualization with large amounts of data
plt.scatter(diamonds['carat'], diamonds['price'], alpha=.1);

In [ ]:
# Linear regression with scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score

# Use the entire dataset
df = diamonds

# Fit the model to the data
model = LinearRegression()
model.fit(df[['carat']], df['price'])

# Evaluate the performance
yhat = model.predict(df[['carat']])
print('Explained variance: {:.2f}%'.format(100 * explained_variance_score(df['price'], yhat)))

In [ ]:
# Plot the model prediction (no noise) as a smooth curve
X_model = [[0], [3]] # dependent variable, no noise
yhat_model = model.predict(X_model) # compute the model prediction

plt.plot(X_model, yhat_model, color='k')

# Plot the data (scatterplot)
plt.scatter(df['carat'], df['price'], alpha=.1)
plt.show();

# Evaluate model performance

Investigate two methods of creating a train/test split.


In [ ]:
# Train-test split with scikit-learn
from sklearn.model_selection import train_test_split

# Train-test split -- split the dataset
def my_train_test_split(X, y, test_size=.3):
    n_training_samples = int((1.0 - test_size) * X.shape[0])

    X_train = X[:n_training_samples,:]
    y_train = y[:n_training_samples]

    X_test = X[n_training_samples:,:]
    y_test = y[n_training_samples:]

    return X_train, X_test, y_train, y_test

# Train/test split

Choose one of two methods:

* Method #1: simple split of the dataset with `my_train_test_split()`
* Method #2: `scikit-learn.model_selection.train_test_split`


In [ ]:
# Sample the entire dataset
X = df['carat'].values.reshape(-1,1)
y = df['price'].values

# Method #1: Train/test split based on sequential sampling of the dataset
X_train, X_test, y_train, y_test = my_train_test_split(X, y, test_size=0.3)

# Method #2: Train/test split with scikit-learn (random sampling)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Plot the model prediction (no noise) as a smooth curve
X_model = [[0], [3]] # dependent variable, no noise
yhat_model = model.predict(X_model) # compute the model prediction
plt.plot(X_model, yhat_model, color='k', label="model")

# Plot the data (scatterplot)
plt.scatter(X_train, y_train, color=['red'], alpha=.1, label="training data")
plt.scatter(X_test, y_test, alpha=.1, label="testing data")
plt.legend();


Data visualization explains the difference between the two methods of train/test split.

Discussion: The results are very sensitive to the way in which the train/test split is implemented with Method #1. This might be interpreted as overfitting. However, overfitting is not what you would expect with a large dataset (more than 50K samples) and a very simple linear-regression model. But data visualization reveals that the sensitivity with Method #1 is really a symptom of model bias combined with non-uniform sampling in the dataset. In fact, with a train/test split based on randomized sampling (Method #2), training and test performance are nearly identical.

Note: To keep things simple, the analysis below doesn't use train/test splits.

In [ ]:
import numpy as np

# Linear regression with polynomials is built-in with Seaborn
# With Scikit-Learn, you need to do it yourself.
from sklearn.preprocessing import PolynomialFeatures

# Add PolynomialFeatures of desired degree
# Inspect X to confirm the result (original series is in column #1
# Column #0 is full of ones (i.e., the intercept)
# To plot a line, use degree=1, in which case X_poly has 2 columns
poly = PolynomialFeatures(degree=5)
X_poly = poly.fit_transform(X)

# Fit the model to the data
# Note: PolynomialFeatures adds the intercept (ones) to column 0
# hence fit_intercept=False, and the data gets moved to column 1
model = LinearRegression(fit_intercept=False)
model.fit(X_poly, y)

# Plot the model prediction (no noise) as a smooth curve
X_model = np.arange(0, 5, .05).reshape(-1, 1) # dependent variable, no noise
X_model = poly.transform(X_model) # add the polynomials
yhat_model = model.predict(X_model) # compute the model prediction
plt.plot(X_model[:,1], yhat_model, color='k') # remember: column 1 is the dependent variable

# Plot the data (scatterplot)
plt.scatter(X[:,0], y, alpha=.5)
plt.show();

# Compute the model prediction at the data points
yhat = model.predict(X_poly)

print('Explained variance: {:.2f}%'.format(100 * explained_variance_score(y, yhat)))

# Residuals

* The simple linear regression model (above) clearly suffers from being oversimplified (high bias).
* Linear regression with a high-order polynomial model has problems as well.
    * The problems are worse where the data are sparse.
* Visualize the residuals to help assess model performance
    * In particular, check for "structure" in residuals that's inconsistent with random noise.

In [ ]:
# Plot the residuals
plt.scatter(X[:,0], y - yhat, alpha=.5)
plt.plot([0, 5], [0,0], linestyle="dashed", color="k")
plt.show();

* If you investigate different values for "degree" in PolynomialFeatures
    * Results:
    * degree = 1 is a linear fit
    * degree > 1 produces strange features where data are sparse
        * but the degree doesn't matter where data are dense
* Investigate the impact of sub-sampling scheme

# Logarithmic dependency?

* Clearly there's structure in the residuals -- is it logarithmic?


# Log scales

* Exponential structure suggests logarithmic scale transformation...
    * ...but `log(x)` isn't helpful in this case
* We can see that by transforming `x` ourselves with matplotlib

In [ ]:
# Log-x scaling for the independent variable (x = carat)
plt.scatter(df['carat'], df['price'], alpha=.1)
plt.gca().set_xscale('log');

* What about `log(y)`?
* Again, this can be visualized with matplotlib

In [ ]:
# Log scaling for the dependent variable (y = price)
plt.scatter(df['carat'], df['price'], alpha=.1)
plt.gca().set_yscale('log');

* And log-log?

In [ ]:
# Data seem relatively linear with log-log scales
plt.scatter(df['carat'], df['price'], alpha=.1)
plt.gca().set_xscale('log')
plt.gca().set_yscale('log');

# Modeling logarithmic dependencies

* log transformation of the data - create two new columns
* transform both feature & target variables for log-log regression
* perform linear regression with log-transformed variables

In [ ]:
# Create a column with log(price) and a column with log(carat)
diamonds['log(price)'] = diamonds['price'].transform(np.log10)
diamonds['log(carat)'] = diamonds['carat'].transform(np.log10)

In [ ]:
# Linear regression with log-transformed data
df = diamonds

X = df[['log(carat)']]
y = df['log(price)']

# Fit the model to the data
model = LinearRegression()
model.fit(X, y)

# Predict the data with the model
yhat = model.predict(X)

# Assign model to a new column in the dataframe
df = df.assign(model_loglog = yhat)

# Evaluate the performance
print('Explained variance: {:.2f}%'.format(100 * explained_variance_score(y, yhat)))

In [ ]:
df

# Residuals with log-log scaling

Is this better?

Be careful about comparing explained variance -- we've rescaled the data.

We need a way to assess the assumption of model + random noise. We'll further analyze the residuals.

In [ ]:
# Plot the residuals
X = df['log(carat)']
residuals = df['log(price)'] - df['model_loglog']

plt.scatter(X, residuals, alpha=.5)
plt.plot([-.7, .7], [0,0], linestyle="dashed", color="k")
plt.show();

# STOPPED HERE


In [ ]:
# Generate some synthetic data: linear model plus random noise
import random
import numpy as np
import matplotlib.pyplot as plt

m = 100

# Model
w0 = 3
w1 = 0.042
xs = np.arange(0, m * 100) / m
ys = w0 + w1 * xs

# Noise
mu, sigma = 0, 0.1 # mean and standard deviation
random.seed(42) # for reproducibility
eps = [random.normalvariate(mu, sigma) for i in enumerate(xs)]

# Data
data = ys + eps

In [ ]:
# Visualization with matplotlib
plt.plot(xs, data,'o', label='data')
plt.plot(xs, ys, label='model')
plt.ylabel('y')
plt.xlabel('x')
plt.legend();

In [ ]:
# Linear regression with scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score

Xs = xs.reshape(-1,1)

print(xs.shape)
print(Xs.shape)

# Fit the model to the data
model = LinearRegression()
model.fit(Xs, data)

# Evaluate the performance
ys_hat = model.predict(Xs)
print('Explained variance: {:.2f}%'.format(100 * explained_variance_score(data, ys_hat)))

In [ ]:
# Visualization with matplotlib
plt.plot(xs, data,'o', label='data')
plt.plot(xs, ys_hat,'--', label='prediction')
plt.plot(xs, ys, label='model')
plt.ylabel('y')
plt.xlabel('x')
plt.legend();

In [ ]:
plt.plot(xs, data - ys_hat, 'o', label='residual')
plt.ylabel('y')
plt.xlabel('x')
plt.legend();

In [ ]:
plt.hist(data);

In [ ]:
plt.hist(data - ys_hat);

# Visualizing data distributions

* Statistical assessment of data and residuals
* Histograms with matplotlib
    * assessing the assumption of random noise
    * looking for [skewness](https://en.wikipedia.org/wiki/Skewness) or other departures from normal distribution
    * styling with the matplotlib API
* Clearly, diamond "price" is right skewed

In [ ]:
from matplotlib.patches import Rectangle

# Histogram of price is highly skewed (positive or right skew)
plt.hist(diamonds['price']);

# Histogram styling. It can be done, but...
ax = plt.gca()
lines = ax.get_lines()
children = ax.get_children()
print('axes:', ax)
print('lines (histograms do not use lines):', lines)
print('children:', type(children), len(children))
print('children[0]:', children[0])
[child.set_edgecolor('k') for child in children if isinstance(child, Rectangle)];

In [ ]:
# Convenience function for styled histograms
def styled_histogram(series):
    plt.hist(series)
    ax = plt.gca()
    children = ax.get_children()
    [child.set_edgecolor('k') for child in children if isinstance(child, Rectangle)];

# Distribution of log(price)

* Compare distributions of price & log(price)
* Compare distributions residuals from log-log model

In [ ]:
# Distribution of log(price) is much more symmetric
styled_histogram(df['log(price)'])

In [ ]:
# Plot the histogram of the residuals from the linear regression of log-log data
residuals = y - yhat
styled_histogram(residuals)

# Q-Q plot

The distributions are symmetric, but are they "good"? And what is a metric for "good"?

* Quantiles (Q-Q plot) of data compared with theoretical probability distributions (probability plot)
* [Quantiles](https://en.wikipedia.org/wiki/Quantile) are values dividing a probability density into equal areas
* Using the scipy.stats library, default theoretical distribution is standard normal
* Using statsmodels, the default distribution is scipy.stats.distributions.norm

### references

* [scipy reference docs](https://docs.scipy.org/doc/scipy/reference/) -- scipy.org
* [scipy.stats.probplot()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html) -- scipy.org



In [ ]:
from scipy import stats

# Q-Q plot (data vs standard normal)
# stats.probplot(offsets[:,1], plot=plt);

## Create a convenience function for a styled Q-Q plot

### styling

`stats.probplot()` uses 2 plots types (line and scatterplot) with harsh colors

* [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot) -- wikipedia
* [scipy.stats.probplot](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html) API reference docs - scipy.org
    * plots quantiles of data against Normal distribution (default)
* Q: Can I change the colors?
* A: Yes, but the `scipy.stats` API docs don't speak to this.
    * You need to use the matplotlib API

In [ ]:
def qqplot(data):
    stats.probplot(data, plot=plt)

    # Change the styling
    ax = plt.gca()
    lines = ax.get_lines() # there are two -- dots and line
    lines[0].set_markerfacecolor("steelblue")
    lines[0].set_markeredgecolor("steelblue")
    lines[0].set_alpha(0.5)
    lines[1].set_color('k');

styled_histogram(residuals)
plt.show()

qqplot(residuals)

# Bad residuals -- linear scales



In [ ]:
X_linear = df[['carat']]
y_linear = df['price']

# Fit the model to the data
model = LinearRegression()
model.fit(X_linear, y_linear)

# Evaluate the performance
yhat_linear = model.predict(X_linear)
residuals_linear = y_linear - yhat_linear
print('Explained variance: {:.2f}%'.format(100 * explained_variance_score(y_linear, yhat_linear)))

In [ ]:
styled_histogram(residuals_linear)
plt.show();

qqplot(residuals_linear)

In [ ]:
# Q-Q plot with statsmodels
import statsmodels.api as sm
sm.qqplot(residuals_linear, fit=True, line="45")

# Change the styling
ax = plt.gca()
lines = ax.get_lines() # there are two -- dots and line
lines[0].set_markerfacecolor("steelblue")
lines[0].set_markeredgecolor("steelblue")
lines[0].set_alpha(0.5)
lines[1].set_color('k');

In [ ]:
# Q-Q plot with statsmodels
import statsmodels.api as sm
sm.qqplot(residuals, fit=True, line="45")

# Change the styling
ax = plt.gca()
lines = ax.get_lines() # there are two -- dots and line
lines[0].set_markerfacecolor("steelblue")
lines[0].set_markeredgecolor("steelblue")
lines[0].set_alpha(0.5)
lines[1].set_color('k');

# EXERCISE

Visualize the relationship between residuals in log-log model and cut, clarity, color
* First, show students how to do a boxplot
* Then let them filter the residuals and visualize relationship

# Visualizing distributions with boxplots

* matplotlib has a boxplot capability (you don't have to use seaborn)
* [boxplot demo](https://matplotlib.org/stable/gallery/pyplots/boxplot_demo_pyplot.html)
* [matplotlib.pyplot.boxplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html) API reference docs

In [ ]:
# Random data
np.random.seed(42)
spread = np.random.rand(50) * 100 # random numbers (mean 50) (50 of them)
center = np.ones(25) * 50  # constants = 50 (25 of them)
flier_high = np.random.rand(10) * 100 + 100 # 10 extra large values
flier_low = np.random.rand(10) * -100 # 10 extra small values
data = np.concatenate((spread, center, flier_high, flier_low))
assert data.shape == (95,)

# Create the boxplot
fig1, ax1 = plt.subplots()
ax1.set_title('Basic Plot')
ax1.boxplot(data);

In [ ]:
# Add more random data 
spread = np.random.rand(50) * 100
center = np.ones(25) * 40
flier_high = np.random.rand(10) * 100 + 100
flier_low = np.random.rand(10) * -100
d2 = np.concatenate((spread, center, flier_high, flier_low))

# Create an array of dataframes, with one dataframe for each subplot
my_list = [data, d2, d2[::2]] # This works, but boxplot wants an array
# my_array = np.array(my_list)  # This conversion issues a deprecation warning
my_array = np.array(my_list, dtype="object") # This is the right way

fig7, ax7 = plt.subplots()
ax7.set_title('Multiple Samples with Different sizes')
ax7.boxplot(my_array); # With "my_list", boxplot issues a deprecation warning

plt.show();

# Other candidate predictors

In [ ]:
print('Unique cuts:', df['cut'].unique())
print('Unique colors:', df['color'].unique())
print('Unique clarities:', df['clarity'].unique())

In [ ]:
df.head()

In [ ]:
# Another visualization of the same data -- Q: Is this more informative?
# It exhibits the non-intuitive relationship between price & cut that we've seen before
import seaborn as sns

cuts = ["Fair", "Good", "Very Good", "Premium", "Ideal"]

sns.catplot(data=df, x="cut", y="price", kind="point", order=cuts);

In [ ]:
# Likewise for log(price) -- relationship to "cut" is hard to understand
sns.catplot(data=df, x="cut", y="log(price)", kind="point", order=cuts);

In [ ]:
# Box plot exhibits similar relationship
sns.catplot(data=df, x="cut", y="log(price)", kind="box", order=cuts);

In [ ]:
df

## Comparing residuals of linear regression

Visualize the relationship between diamond "cut" and residuals of linear model (after log-log transormation).

In contrast with visualization above, residuals exhibit the expected relationahip to cut:

Higher cut quality is asociated with larger log(price) 

In [ ]:
df['resid'] = df['log(price)'] - df['model_loglog']
df

sns.catplot(data=df, x="cut", y="resid", kind="box", order=cuts);

# Linear regression with categorical features - one-hot encoding

* You *can* do linear regression with categorical features
  * one-hot encoding of categorical features
  * use `drop="first"` (to avoid colinear inputs)
* predict log(price)
* add log(carat) as a feature
* [6.3 Preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) (sklearn.preprocessing package) -- scikit-learn.org
    * [6.3.4 Encoding categorical features](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features) -- scikit-learn.org
    * One-hot encoding of categorical features (indicator variables)
* Documentation discusses `drop="feature"` parameter that avoids colinear inputs
    * Colinear inputs would cause non-regularized linear regression to fail

### modeling the residuals

* the next cell uses categorical features to model the residuals of linear regression with log-log transformation
  * first model -- log(price) vs log(carat)
  * this is a type of "forward selection"

In [ ]:
# Multivariate linear regression of residuals from log-log model
# Uses one-hot encoding of the categorical features
from sklearn import preprocessing
from scipy.sparse import hstack
from sklearn.metrics import explained_variance_score

enc = preprocessing.OneHotEncoder(drop="first")
#enc = preprocessing.OneHotEncoder()

y = df['resid']
X = df.loc[:, ['cut', 'color', 'clarity']].values

X = enc.fit_transform(X)

model = LinearRegression()
model.fit(X, y)

yhat = model.predict(X)

print('Explained variance (before): {:.2f}%'.format(100 * explained_variance_score(df['log(price)'], df['model_loglog'])))
print('Explained variance (after): {:.2f}%'.format(100 * explained_variance_score(df['log(price)'], df['model_loglog'] + yhat)))

# Simultaneous multivariate regression of log(price)

* rather than a sequence of models this model considers all variables simultaneously
* explained variance is larger than the result from the cell above

In [ ]:
# Multivariate linear regression with one-hot encoding of categorical features
# This model includes log(carat) as a variable in the model
from sklearn import preprocessing
from scipy.sparse import hstack
from sklearn.metrics import explained_variance_score

enc = preprocessing.OneHotEncoder(drop="first")
#enc = preprocessing.OneHotEncoder()

y = df['log(price)']
X = df.loc[:, ['cut', 'color', 'clarity']].values

X = enc.fit_transform(X)

print(type(X))
print("X.shape:", X.shape)
print(len(enc.categories_))
print(enc.categories_)
print('df[log(carat)].shape', df['log(carat)'].values.reshape(-1,1).shape)

X = hstack((X, df[['log(carat)']].values))

model = LinearRegression()
model.fit(X, y)

yhat = model.predict(X)

print('Explained variance (before): {:.2f}%'.format(100 * explained_variance_score(y, df['model_loglog'])))
print('Explained variance (after): {:.2f}%'.format(100 * explained_variance_score(y, yhat)))